<a href="https://colab.research.google.com/github/Inkvizition/Try_to_AI/blob/main/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [ ]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()


--2021-02-07 09:30:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2021-02-07 09:30:15 (194 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy1-05.png', 'happy2-07.png', 'happy1-07.png', 'happy2-06.png', 'happy1-16.png', 'happy2-09.png', 'happy2-13.png', 'happy2-02.png', 'happy2-18.png', 'happy1-13.png']
['sad2-09.png', 'sad1-08.png', 'sad1-00.png', 'sad2-07.png', 'sad2-10.png', 'sad2-17.png', 'sad2-13.png', 'sad2-11.png', 'sad2-08.png', 'sad1-05.png']


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('acc')>=0.998):
          print("Достигнута точность 99.9%, поэтому обучение закончено!")
          self.model.stop_training = True
def train_happy_sad_model():
    DESIRED_ACCURACY = 0.999


    callbacks = myCallback()
    
    # Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])
    

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Используйте target_size  150 X 150.
    train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(150, 150),
        batch_size=128,
        class_mode='binary')

    history = model.fit(
      train_generator, 
      epochs=20,
      callbacks = [callbacks])
    # model fitting
    return history.history['acc'][-1]

In [ ]:
# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6904 - acc: 0.5000
Epoch 2/20
1/1 [==============================] - 1s 1s/step - loss: 0.5667 - acc: 0.9375
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 4.2381 - acc: 0.5000
Epoch 4/20
1/1 [==============================] - 1s 1s/step - loss: 0.8868 - acc: 0.5000
Epoch 5/20
1/1 [==============================] - 1s 1s/step - loss: 0.5614 - acc: 0.7500
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 0.5132 - acc: 1.0000
Достигнута точность 99.9%, поэтому обучение закончено!


1.0